In [66]:
import datetime as datetime
import pandas as pd
import math
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pyodbc
import requests

short_start = "2019-2-01"
med_start = "2019-1-01"
long_start = "2018-01-01"
short_end = "2019-3-01"
med_end = "2019-3-01"
long_end = "2019-3-01"

drop_list = ['XRP' , 'XLM' , 'BCH' , 'LTC' , 'ADA' , 'TRX' , 'EOS' , 'BSV']
long_drop = ['currency' , 'platform' , 'application']

tfs = ['short' , 'med' , 'long']

df_all = dict.fromkeys(tfs)
df_benchmarks = dict.fromkeys(tfs)
returns_all = dict.fromkeys(tfs)
returns_benchmarks = dict.fromkeys(tfs)
results = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
net_return = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
ann_factor = dict.fromkeys(tfs)
total_return = dict.fromkeys(tfs)
percent_sd = dict.fromkeys(tfs)
summary = dict.fromkeys(tfs)
corr = dict.fromkeys(tfs)
betas = dict.fromkeys(tfs)
beta_results = dict.fromkeys(tfs)
total_return_excel = dict.fromkeys(tfs)
sortino_df = dict.fromkeys(tfs)
sortino = dict.fromkeys(tfs)
average_return = dict.fromkeys(tfs)

data_bletchley = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\\bletchley_history.xlsx' , index_col= 0)
data_coins = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bletchley.xlsx' , index_col=0)
data_bletchley['ERC20'] = data_bletchley['ethereum']
data_bletchley = data_bletchley.drop('ethereum' , axis=1)
data_bletch = data_bletchley

In [67]:
data_finance = data_bletch.fillna(method='ffill')
data_finance = data_finance.dropna()
index_data = data_finance
data_coins = data_coins.fillna(method='ffill')
data_coins = data_coins.dropna()

index_data.index = (pd.to_datetime(index_data.index.values,unit='ms'))
index_data.index = index_data.index.round('d')
df = index_data.join(data_coins, how='inner')

returns = df.pct_change()
returns = returns.dropna()

In [68]:
#filter master dataframes based on date ranges
returns_all['short'] = returns.loc[short_start:short_end]
returns_benchmarks['short'] = returns.loc[short_start:short_end]
df_all['short'] = df.loc[short_start:short_end]
df_benchmarks['short'] = df.loc[short_start:short_end]

returns_all['med'] = returns.loc[med_start:med_end]
returns_benchmarks['med'] = returns.loc[med_start:med_end]
df_all['med'] = df.loc[med_start:med_end]
df_benchmarks['med'] = df.loc[med_start:med_end]

returns_all['long'] = returns.loc[long_start:long_end]
returns_benchmarks['long'] = returns.loc[long_start:long_end]
df_all['long'] = df.loc[long_start:long_end]
df_benchmarks['long'] = df.loc[long_start:long_end]

#drop coins that aren't benchmarks
for c in drop_list:
    for t in tfs:
        returns_benchmarks[t] = returns_benchmarks[t].drop([c] , axis=1)
        df_benchmarks[t] = df_benchmarks[t].drop([c] , axis=1)

In [69]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,-0.002724,0.026888
2019-02-27,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.002282,-0.005211
2019-02-28,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,0.006208,-0.011987
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.002092,0.005055


In [70]:
#drop sectors that don't have data back to 1/1/2018
for i in long_drop:
    df_benchmarks['long'] = df_benchmarks['long'].drop([i] , axis=1)
    returns_benchmarks['long'] = returns_benchmarks['long'].drop([i] , axis=1)

In [71]:
#calculate sortino to btc
for t in tfs:
    sortino_df[t] = pd.DataFrame()
    sortino[t] = pd.DataFrame(index = ['sortino'])
    for col in returns_benchmarks[t]:
        sortino_df[t][col] = returns_benchmarks[t][col] - returns_benchmarks[t]['BTC']
        avg_ret = np.mean(sortino_df[t][col])
        a = [r ** 2 for r in sortino_df[t][col] if r < 0]
        a_sum = np.sum(a)
        b = len(sortino_df[t][col])
        down_risk = np.sqrt(a_sum/b)
        if down_risk > 0.000001:
            sortino[t][col] = (avg_ret / down_risk)*math.sqrt(365)
        else:
            sortino[t][col] = 0

In [72]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,-0.002724,0.026888
2019-02-27,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.002282,-0.005211
2019-02-28,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,0.006208,-0.011987
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.002092,0.005055


In [73]:
data_coins_week = data_coins.loc[short_start:short_end]
#data_coins_week = data_coins_week[:]
net_week_return = data_coins_week.pct_change(len(data_coins_week)-1)
ann_week_factor = 365.000/(len(data_coins_week))
data_coins_week_returns = pd.DataFrame()
data_coins_week_returns = data_coins_week.pct_change()
data_coins_week_returns = data_coins_week_returns.dropna()
total_week_return = net_week_return.tail(1)*100
data_coins_week = data_coins_week.append(total_week_return)
data_coins_week.index = data_coins_week.index[:-1].append(pd.Index(['return']))

In [74]:
data_coins_week

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
2019-02-01 00:00:00,3434.159070,106.940900,0.310080,0.081510,31.582380,113.855540,0.025150,2.320780,0.038370,63.784020
2019-02-02 00:00:00,3504.075450,107.600930,0.308830,0.081560,32.921310,115.791230,0.026350,2.375950,0.038890,64.382450
2019-02-03 00:00:00,3462.765390,109.470430,0.305300,0.081340,34.056510,119.396750,0.025570,2.391120,0.038550,64.364290
2019-02-04 00:00:00,3438.310100,106.533190,0.299450,0.079410,33.163810,117.557760,0.025170,2.358450,0.037850,63.267440
2019-02-05 00:00:00,3429.255370,106.857480,0.297330,0.078050,33.872030,117.918550,0.026700,2.385360,0.037690,62.616330
2019-02-06 00:00:00,3434.783920,106.494340,0.293810,0.075640,33.681330,115.625790,0.025780,2.344850,0.037340,60.059600
2019-02-07 00:00:00,3387.866700,104.071050,0.286960,0.073410,32.492230,113.693800,0.025270,2.330850,0.036370,62.074720
2019-02-08 00:00:00,3478.950920,106.282050,0.304710,0.077920,34.602420,117.799570,0.027300,2.454950,0.038490,65.547730
2019-02-09 00:00:00,3635.724300,118.499660,0.309310,0.080520,42.681750,128.417160,0.026500,2.698510,0.040350,66.661070
2019-02-10 00:00:00,3699.248260,119.028000,0.311240,0.080520,44.650460,127.399090,0.026190,2.794000,0.041060,65.832840


In [75]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.002041,-0.011593
2019-02-02,0.010041,0.011111,0.008623,0.010051,0.011235,0.004722,0.028551,0.030775,0.020359,0.006172
2019-02-03,0.015933,0.004272,0.006784,0.014922,0.014239,0.014904,0.012689,0.005754,-0.011789,0.017374
2019-02-04,-0.020945,-0.015801,-0.003981,-0.020153,-0.018528,-0.023694,-0.009049,-0.013480,-0.007062,-0.026831
2019-02-05,-0.003668,0.000808,-0.013358,-0.003647,-0.004557,-0.007102,0.018634,0.026514,-0.002633,0.003044
2019-02-06,0.001495,0.017993,-0.017855,0.002038,0.002691,-0.002001,0.016667,0.009467,0.001612,-0.003398
2019-02-07,-0.017216,-0.016160,-0.016814,-0.017138,-0.015711,-0.025014,0.001640,-0.006391,-0.013659,-0.022755
2019-02-08,-0.005118,0.004567,0.007774,-0.004129,-0.005254,0.001828,-0.008564,-0.007530,0.026885,0.021245
2019-02-09,0.099001,0.089244,0.074505,0.097697,0.093065,0.110015,0.069554,0.065363,0.045063,0.114955
2019-02-10,-0.001943,0.010255,0.007582,-0.000879,-0.002645,0.005028,0.015300,0.007550,0.017472,0.004459


In [76]:
print ('short start: ', returns_benchmarks['short'].index[0])
print ('medium start: ', returns_benchmarks['med'].index[0])
print ('long start: ', returns_benchmarks['long'].index[0])
print ('end: ', returns_benchmarks['short'].index[-2])
print ('end: ', returns_benchmarks['med'].index[-2])
print ('end: ', returns_benchmarks['long'].index[-2])

short start:  2019-02-01 00:00:00
medium start:  2019-01-01 00:00:00
long start:  2018-11-09 00:00:00
end:  2019-02-28 00:00:00
end:  2019-02-28 00:00:00
end:  2019-02-28 00:00:00


In [77]:
returns_benchmarks['short'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-01,-0.015413,-0.038908,-0.040316,-0.017795,-0.010820,-0.030957,-0.038853,-0.044541,-0.002041,-0.011593
2019-02-02,0.010041,0.011111,0.008623,0.010051,0.011235,0.004722,0.028551,0.030775,0.020359,0.006172
2019-02-03,0.015933,0.004272,0.006784,0.014922,0.014239,0.014904,0.012689,0.005754,-0.011789,0.017374
2019-02-04,-0.020945,-0.015801,-0.003981,-0.020153,-0.018528,-0.023694,-0.009049,-0.013480,-0.007062,-0.026831
2019-02-05,-0.003668,0.000808,-0.013358,-0.003647,-0.004557,-0.007102,0.018634,0.026514,-0.002633,0.003044


In [78]:
print (data_coins_week.head(n=1))
data_coins_week.tail(n=1)

                            BTC       ETH      XRP      XLM       LTC  \
2019-02-01 00:00:00  3434.15907  106.9409  0.31008  0.08151  31.58238   

                           BCH      TRX      EOS      ADA       BSV  
2019-02-01 00:00:00  113.85554  0.02515  2.32078  0.03837  63.78402  


,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
return,11.229919,27.113995,1.119066,3.729604,44.293749,14.095072,-5.964215,51.014745,11.780036,4.309449


In [79]:
annualize = math.sqrt(365)
for t in tfs:
    results[t] = returns_benchmarks[t].describe()
    vol[t] = pd.DataFrame()
    vol[t] = results[t].loc['std']*annualize*100
    vol[t].name = 'annualized volatility'
    results[t] = results[t].append(df_benchmarks[t].iloc[0])
    results[t].index = results[t].index[:-1].append(pd.Index(['open price']))
    results[t] = results[t].append(df_benchmarks[t][-1:])
    results[t].index = results[t].index[:-1].append(pd.Index(['last price']))
    results[t] = results[t].append(vol[t])

In [80]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-25,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,-0.002724,0.026888
2019-02-27,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.002282,-0.005211
2019-02-28,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,0.006208,-0.011987
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.002092,0.005055


In [81]:
for t in tfs:
    average_return[t] = pd.DataFrame()
    average_return[t] = returns_benchmarks[t]
    average_return[t].loc['mean'] = returns_benchmarks[t].mean(axis=0)
    returns_benchmarks[t] = returns_benchmarks[t][:-1]

In [82]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-25 00:00:00,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26 00:00:00,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,-0.002724,0.026888
2019-02-27 00:00:00,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.002282,-0.005211
2019-02-28 00:00:00,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,0.006208,-0.011987
2019-03-01 00:00:00,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.002092,0.005055


In [83]:
for t in tfs:
    net_return[t] = df_benchmarks[t].pct_change(len(df_benchmarks[t])-1)
    ann_factor[t] = 365.000/(len(df_benchmarks[t]))
    total_return[t] = net_return[t].tail(1)*100
    total_return_excel[t] = net_return[t].tail(1)
    percent_sd[t] = results[t].iloc[2,:]*100
    results[t] = results[t].append(percent_sd[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['daily volatility']))
    #print total_return_excel[t]
    results[t] = results[t].append(total_return_excel[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['return']))
    #print results[t]
    #print average_return[t]/(percent_sd[t]*0.01).transpose()
    Sharpe_values = (average_return[t].loc['mean']/(percent_sd[t]*0.01))*annualize
    Sharpe_values = Sharpe_values.rename('sharpe_values')
    results[t] = results[t].append(Sharpe_values)
    results[t].index = results[t].index[:-1].append(pd.Index(['Sharpe']))
    results[t] = results[t].append(sortino[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['Sortino']))
#print (results[t])

In [84]:
returns_benchmarks['med'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-01-01 00:00:00,-0.046381,-0.050800,-0.056148,-0.046939,-0.042739,-0.057864,-0.040122,-0.046808,-0.012938,-0.038211
2019-01-02 00:00:00,0.046762,0.043521,0.042677,0.046447,0.043137,0.051576,0.036109,0.039667,0.047316,0.058320
2019-01-03 00:00:00,0.033371,0.035948,0.027412,0.033396,0.022256,0.053596,0.031989,0.027348,-0.019799,0.083028
2019-01-04 00:00:00,-0.041354,-0.043182,-0.034871,-0.041317,-0.036859,-0.054475,-0.028133,-0.029247,-0.000386,-0.027875
2019-01-05 00:00:00,0.018170,0.025959,0.024532,0.018870,0.013260,0.025122,0.048155,0.050412,0.002143,0.040172


In [85]:
for t in tfs:
    summary[t] = results[t].iloc[8:]
    corr[t] = returns_benchmarks[t].corr(method='spearman')
    betas[t] = returns_benchmarks[t].cov()
    beta_results[t] = pd.DataFrame()
    beta_results[t] = betas[t].iloc[0,:] / betas[t].iloc[3,0]
    corr[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_corr'+t+'.csv')
    summary[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_results'+t+'.csv')
beta_all = pd.DataFrame()
beta_all['MTD'] = beta_results['short']
beta_all['QTD'] = beta_results['med']
beta_all['YTD'] = beta_results['long']

In [86]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-02-25 00:00:00,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.019460,-0.128770
2019-02-26 00:00:00,0.024443,0.002184,0.021208,0.022781,0.020215,0.026317,0.006748,0.004799,-0.002724,0.026888
2019-02-27 00:00:00,-0.011050,-0.017926,0.013482,-0.010856,-0.007488,-0.021963,-0.001874,-0.003760,-0.002282,-0.005211
2019-02-28 00:00:00,0.001516,0.010927,0.001616,0.002168,0.003785,0.000559,0.015061,0.012899,0.006208,-0.011987
2019-03-01 00:00:00,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.002092,0.005055


In [87]:
summary['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,374.603942,313.478539,49.263316,156.644587,20.150812,13.096591,14.062167,43.673755,3434.159070,106.940900
last price,424.914158,381.398019,57.301770,178.648490,22.516762,15.764553,17.515368,50.875261,3819.812360,135.936850
annualized volatility,67.025544,62.029031,56.191178,66.002242,60.891152,82.038591,52.589428,52.100334,38.151922,83.226787
daily volatility,3.508277,3.246748,2.941181,3.454715,3.187189,4.294096,2.752656,2.727056,1.996963,4.356289
return,0.134302,0.216664,0.163173,0.140470,0.117412,0.203714,0.245567,0.164893,0.112299,0.271140
Sharpe,2.476499,3.594846,2.830471,2.564431,2.440604,2.849430,4.730093,2.936254,3.756348,3.991633
Sortino,0.557995,2.157993,0.465785,0.650680,0.143084,1.897643,3.232488,0.278084,0.000000,4.205596


In [88]:
summary['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,420.511599,337.862280,54.644247,175.290083,22.237102,15.347474,13.168780,39.924454,3773.237410,134.063560
last price,424.914158,381.398019,57.301770,178.648490,22.516762,15.764553,17.515368,50.875261,3819.812360,135.936850
annualized volatility,65.777609,72.747511,67.634188,65.839665,59.762934,83.248551,71.696154,74.091912,38.755181,83.157289
daily volatility,3.442957,3.807779,3.540135,3.446206,3.128135,4.357428,3.752748,3.878148,2.028539,4.352651
return,0.010470,0.128856,0.048633,0.019159,0.012576,0.027176,0.330068,0.274288,0.012343,0.013973
Sharpe,-0.021956,0.949376,0.245105,0.053790,-0.026823,0.171712,2.466828,1.990653,0.178812,0.225191
Sortino,-0.217884,1.439052,0.241385,-0.088029,-0.249688,0.147664,4.260556,3.404879,0.000000,0.249212


In [89]:
summary['long']

,ten,twenty,forty,total,ERC20,BTC,ETH
open price,740.953740,637.900186,110.287135,311.303644,73.888661,6496.416980,216.812750
last price,424.914158,381.398019,57.301770,178.648490,50.875261,3819.812360,135.936850
annualized volatility,85.369336,96.667502,99.162240,86.135318,99.542633,63.786464,98.773786
daily volatility,4.468435,5.059808,5.190389,4.508529,5.210300,3.338736,5.170056
return,-0.426531,-0.402104,-0.480431,-0.426128,-0.311461,-0.412012,-0.373022
Sharpe,-1.690428,-1.243088,-1.644315,-1.664784,-0.719295,-2.386417,-1.048266
Sortino,0.232244,0.770730,-0.252514,0.256598,1.913906,0.000000,1.090981


In [90]:
corr['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.863711,0.843459,0.995621,0.991242,0.959496,0.765736,0.795840,0.227148,0.894910
twenty,0.863711,1.000000,0.710454,0.873016,0.859880,0.902573,0.876300,0.872469,0.347017,0.779420
forty,0.843459,0.710454,1.000000,0.851122,0.829228,0.879584,0.638752,0.653530,0.331691,0.865353
total,0.995621,0.873016,0.851122,1.000000,0.987958,0.966612,0.794745,0.820471,0.227148,0.901478
currency,0.991242,0.859880,0.829228,0.987958,1.000000,0.940339,0.777230,0.808976,0.269294,0.874111
platform,0.959496,0.902573,0.879584,0.966612,0.940339,1.000000,0.766284,0.783799,0.236453,0.922277
application,0.765736,0.876300,0.638752,0.794745,0.777230,0.766284,1.000000,0.986864,0.395731,0.665025
ERC20,0.795840,0.872469,0.653530,0.820471,0.808976,0.783799,0.986864,1.000000,0.371648,0.680898
BTC,0.227148,0.347017,0.331691,0.227148,0.269294,0.236453,0.395731,0.371648,1.000000,0.267652
ETH,0.894910,0.779420,0.865353,0.901478,0.874111,0.922277,0.665025,0.680898,0.267652,1.000000


In [91]:
corr['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.900468,0.893863,0.997779,0.994565,0.978317,0.817300,0.814670,0.266102,0.825716
twenty,0.900468,1.000000,0.830918,0.915371,0.895675,0.929632,0.932963,0.934775,0.327878,0.748042
forty,0.893863,0.830918,1.000000,0.902338,0.889714,0.914027,0.792577,0.775862,0.320397,0.786207
total,0.997779,0.915371,0.902338,1.000000,0.992051,0.982408,0.841379,0.838282,0.267329,0.828755
currency,0.994565,0.895675,0.889714,0.992051,1.000000,0.965459,0.817241,0.814027,0.282350,0.808825
platform,0.978317,0.929632,0.914027,0.982408,0.965459,1.000000,0.832437,0.826417,0.257744,0.837580
application,0.817300,0.932963,0.792577,0.841379,0.817241,0.832437,1.000000,0.987493,0.373524,0.691584
ERC20,0.814670,0.934775,0.775862,0.838282,0.814027,0.826417,0.987493,1.000000,0.350321,0.682992
BTC,0.266102,0.327878,0.320397,0.267329,0.282350,0.257744,0.373524,0.350321,1.000000,0.302572
ETH,0.825716,0.748042,0.786207,0.828755,0.808825,0.837580,0.691584,0.682992,0.302572,1.000000


In [92]:
corr['long']

,ten,twenty,forty,total,ERC20,BTC,ETH
ten,1.000000,0.931706,0.941427,0.998838,0.879301,0.648862,0.849720
twenty,0.931706,1.000000,0.918577,0.942213,0.952566,0.632444,0.833635
forty,0.941427,0.918577,1.000000,0.948372,0.884811,0.674112,0.849241
total,0.998838,0.942213,0.948372,1.000000,0.892490,0.648512,0.853624
ERC20,0.879301,0.952566,0.884811,0.892490,1.000000,0.652228,0.771321
BTC,0.648862,0.632444,0.674112,0.648512,0.652228,1.000000,0.557351
ETH,0.849720,0.833635,0.849241,0.853624,0.771321,0.557351,1.000000


In [93]:
beta_all

,MTD,QTD,YTD
ten,1.015897,0.999704,0.991443
twenty,0.870463,1.021342,1.078920
forty,0.791862,0.945079,1.100599
total,1.000000,1.000000,1.000000
currency,0.920786,0.902802,NaN
platform,1.225779,1.246156,NaN
application,0.683329,0.904314,NaN
ERC20,0.676321,0.913469,1.052608
BTC,0.217382,0.292723,0.597361
ETH,1.149069,1.118268,1.012332


In [94]:
coins_high_low = data_coins_week_returns.describe()
coins_high_low

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,0.004004,0.009526,0.000755,0.001825,0.014759,0.005599,-0.001540,0.016133,0.004368,0.002307
std,0.020192,0.043653,0.027404,0.032937,0.059017,0.042694,0.036700,0.052795,0.028227,0.041006
min,-0.045528,-0.128770,-0.056299,-0.066581,-0.089641,-0.127891,-0.104960,-0.120839,-0.054752,-0.089860
25%,-0.006086,-0.007437,-0.015914,-0.018777,-0.011366,-0.010999,-0.020927,-0.012953,-0.010544,-0.016156
50%,0.000901,0.005614,-0.003346,-0.000065,0.018738,-0.000734,-0.008355,0.014700,0.000457,-0.005020
75%,0.018194,0.022522,0.010873,0.009898,0.035546,0.022805,0.017152,0.029484,0.021341,0.020676
max,0.045063,0.114955,0.076641,0.087388,0.233490,0.126676,0.080332,0.179601,0.058290,0.102308


In [95]:
returns_benchmarks['short'].describe()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,0.004548,0.006109,0.004357,0.004637,0.004072,0.006404,0.006815,0.004191,0.003926,0.009102
std,0.035083,0.032467,0.029412,0.034547,0.031872,0.042941,0.027527,0.027271,0.019970,0.043563
min,-0.105154,-0.083468,-0.088509,-0.103215,-0.094980,-0.122560,-0.074005,-0.070627,-0.045528,-0.128770
25%,-0.011183,-0.009923,-0.010825,-0.011053,-0.008193,-0.016901,-0.008685,-0.009017,-0.004727,-0.007437
50%,0.001248,0.004419,0.007192,0.002103,0.001441,0.002737,0.014040,0.005451,0.000901,0.005614
75%,0.016787,0.022655,0.019059,0.017700,0.014487,0.025802,0.018508,0.021403,0.016734,0.021368
max,0.099001,0.089244,0.074505,0.097697,0.093065,0.110015,0.069554,0.065363,0.045063,0.114955


In [96]:
data_coins_week.tail(n=1)

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV
return,11.229919,27.113995,1.119066,3.729604,44.293749,14.095072,-5.964215,51.014745,11.780036,4.309449
